In [45]:
import pandas as pd
import numpy as np
import requests
import json
from pandas_ods_reader import read_ods

In [46]:
# retrieve data from api on the web
url = 'http://chargepoints.dft.gov.uk/api/retrieve/registry/format/csv'
chargepoint_df = pd.read_csv(url)

/var/folders/j9/0qd6cw014pgb7rk0v_28prrh0000gn/T/ipykernel_4110/372998491.py:3: DtypeWarning: Columns (5,10,11,16,17,18,54,56,57,58,59,60,61,62,63,64,65,66,67,68,69,79,90,101,112,115,119,122,123,126,130,133,134,137,141,144,148,152,155) have mixed types. Specify dtype option on import or set low_memory=False.
  chargepoint_df = pd.read_csv(url)


In [47]:
# get shape of data
chargepoint_df.head()

,chargeDeviceID,reference,name,latitude,longitude,subBuildingName,buildingName,buildingNumber,thoroughfare,street,...,connector8Type,connector8RatedOutputKW,connector8OutputCurrent,connector8RatedVoltage,connector8ChargeMethod,connector8ChargeMode,connector8TetheredCable,connector8Status,connector8Description,connector8Validated
0,c911241d00294e8bb714eee2e83fa475,PP-12289,Alex F Noble & Son,55.875053,-3.173333,NaN,NaN,NaN,NaN,Swinton Place,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,fa6c94460e902005a0b660266190c8ba,PP-12295,Ancaster Nissan Dealership,51.411173,-0.055369,NaN,NaN,61,Croydon Road,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,eb1848290d5a7de9c9ccabc67fefa211,PP-12290,Beadles Nissan Ltd,51.451127,0.050619,NaN,NaN,43-53,Eltham High Street,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,91e50fe1e39af2869d3336eaaeebdb43,PP-12292,Benfield Motors,54.978947,-1.599306,NaN,NaN,176,Portland Road,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,65b1e92c585fd4c2159d5f33b5030ff2,PP-12198,Circus Road,51.533633,-0.172353,NaN,NaN,NaN,Circus Road,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
chargepoint_df.shape

(31664, 158)

In [49]:



# retreive comments from openchargemap api
url = 'https://api.openchargemap.io/v3/poi/?output=json&countrycode=GB&maxresults=100000&compact=true&verbose=false&key=1e2b0b1e-5b1e-4b3a-8b9a-0b8b9a3a2b1e&includecomments=true'
response = requests.get(url)
data = response.json()

# convert json to dataframe
ocm_comments_df = pd.DataFrame(data)
ocm_comments_df.to_csv('data/ocm_comments.csv')



In [50]:
ocm_comments_df.shape

(25332, 23)

In [51]:
ocm_comments_df.head()
# unnest the column AddressInfo and UserComments
ocm_comments_df = pd.concat([ocm_comments_df.drop(['AddressInfo'], axis=1), ocm_comments_df['AddressInfo'].apply(pd.Series)], axis=1)

In [52]:
# join the two dataframes using latitude and longitude
chargepoint_df = chargepoint_df.merge(ocm_comments_df, how='left', left_on=['latitude', 'longitude'], right_on=['Latitude', 'Longitude'])

In [53]:
chargepoint_df.head()

# save the data to csv
chargepoint_df.to_csv('data/chargepoint.csv')

In [112]:
# get electricity data with only sheet name "2.2.4" and skipping first 12 rows
electricity_df = pd.read_excel('data/table_224.xlsx', sheet_name='2.2.4', skiprows=12)
# rename the columns
electricity_df.columns = ['Year', 'Region', 'PES area', '1', '2', '3', '4', '5', '6', '7', 'Average fixed cost (£/year)']
# drop columns 1-7
electricity_df = electricity_df.drop(['1', '2', '3', '4', '5', '6', '7'], axis=1)
# drop Region column
electricity_df = electricity_df.drop(['Region'], axis=1)
electricity_df.head()

,Year,PES area,Average fixed cost (£/year)
0,2010,Northern Scotland,48.789916
1,2010,Northern Ireland,0.000000
2,2010,West Midlands,34.616904
3,2010,South East,45.487996
4,2010,South Wales,39.022561


In [113]:
# show unique values in PES area
electricity_df['PES area'].unique()

array(['Northern Scotland', 'Northern Ireland', 'West Midlands',
       'South East', 'South Wales', 'Southern Scotland', 'Eastern',
       'Yorkshire', 'Merseyside & North Wales', 'London', 'North West',
       'North East', 'East Midlands', 'South West', 'Southern',
       'United Kingdom', 'North Scotland', 'South Scotland'], dtype=object)

In [116]:
# replace PES area values with the correct names

electricity_df['PES area'] = electricity_df['PES area'].replace(['Northern Scotland'], 'North Scotland')
electricity_df['PES area'] = electricity_df['PES area'].replace(['Southern Scotland'], 'South Scotland')
# check if the values are replaced
electricity_df['PES area'].unique()

array(['North Scotland', 'Northern Ireland', 'West Midlands',
       'South East', 'South Wales', 'South Scotland', 'Eastern',
       'Yorkshire', 'Merseyside & North Wales', 'London', 'North West',
       'North East', 'East Midlands', 'South West', 'Southern',
       'United Kingdom'], dtype=object)

In [131]:
# population data
population_df = pd.read_csv('data/ukdetailedtimeseries2001to2020/MYEB1_detailed_population_estimates_series_UK_(2020_geog21).csv')
# remove word "population" from column names
population_df.columns = population_df.columns.str.replace('population_', '')
population_df

,ladcode21,laname21,country,sex,age,2001,2002,2003,2004,2005,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,E06000001,Hartlepool,E,1,0,525,502,516,553,531,...,601,595,576,551,497,576,507,535,515,517
1,E06000001,Hartlepool,E,1,1,544,526,498,526,550,...,573,615,601,567,553,512,570,515,532,519
2,E06000001,Hartlepool,E,1,2,548,548,551,507,521,...,580,586,621,600,576,562,534,576,515,528
3,E06000001,Hartlepool,E,1,3,556,546,548,553,508,...,600,578,584,625,598,579,565,530,583,520
4,E06000001,Hartlepool,E,1,4,606,549,553,540,554,...,600,616,593,568,620,604,591,576,533,580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68063,W06000024,Merthyr Tydfil,W,2,86,94,93,92,101,106,...,124,103,100,117,121,113,118,90,127,103
68064,W06000024,Merthyr Tydfil,W,2,87,91,83,83,80,86,...,101,113,92,85,108,107,106,119,78,108
68065,W06000024,Merthyr Tydfil,W,2,88,88,82,70,76,74,...,93,92,94,78,72,92,96,90,100,67
68066,W06000024,Merthyr Tydfil,W,2,89,56,71,67,53,65,...,87,87,78,77,67,55,77,85,88,97


In [132]:
# melt the dataframe
population_df = pd.melt(population_df, id_vars=['ladcode21', 'laname21','country','sex','age'], var_name='Year', 
                        value_vars=['2001', '2002', '2003', '2004', '2005', '2006', '2007','2008', '2009', '2010', '2011', '2012', 
                                    '2013','2014', '2015', '2016', '2017', '2018', '2019', '2020'])
# rename the column value to population
population_df = population_df.rename(columns={'value': 'population'})
population_df

,ladcode21,laname21,country,sex,age,year,population
0,E06000001,Hartlepool,E,1,0,2001,525
1,E06000001,Hartlepool,E,1,1,2001,544
2,E06000001,Hartlepool,E,1,2,2001,548
3,E06000001,Hartlepool,E,1,3,2001,556
4,E06000001,Hartlepool,E,1,4,2001,606
...,...,...,...,...,...,...,...
1361355,W06000024,Merthyr Tydfil,W,2,86,2020,103
1361356,W06000024,Merthyr Tydfil,W,2,87,2020,108
1361357,W06000024,Merthyr Tydfil,W,2,88,2020,67
1361358,W06000024,Merthyr Tydfil,W,2,89,2020,97


In [133]:
# gdhi data
gdhi_df = pd.read_excel("data/regionalgrossdisposablehouseholdincomeallitlregions.xls", sheet_name="Table 3", skiprows=1)
# melt the dataframe
gdhi_df = pd.melt(gdhi_df, id_vars=['Region name', 'ITL level','ITL code'], var_name='Year', value_vars=['1997', '1998', 
                                                                                                         '1999', '2000', '2001', '2002', 
                                                                                                         '2003','2004', '2005', '2006', 
                                                                                                         '2007', '2008', '2009', '2010',
                                                                                                         '2011', '2012', '2013', '2014', 
                                                                                                         '2015', '2016', '2017','2018', 
                                                                                                         '2019', '2020'])
gdhi_df.head()

,Region name,ITL level,ITL code,Year,value
0,United Kingdom,UK,UK,1997,10757
1,England,Other,TLB,1997,11016
2,North East,ITL1,TLC,1997,9253
3,Tees Valley and Durham,ITL2,TLC1,1997,9200
4,Hartlepool and Stockton-on-Tees,ITL3,TLC11,1997,9264


In [134]:
path = "data/veh0105.ods"

# load a sheet based on its index (1 based)
sheet_idx = 4
vehicle_fuel_df = read_ods(path, sheet_idx)

# make third row the header
vehicle_fuel_df.columns = vehicle_fuel_df.iloc[3]
# drop first 4 rows
vehicle_fuel_df = vehicle_fuel_df.iloc[4:]
vehicle_fuel_df.head()

# renumber index
vehicle_fuel_df = vehicle_fuel_df.reset_index(drop=True)

# melt the dataframe
vehicle_fuel_df = pd.melt(vehicle_fuel_df, id_vars=['Units', 'BodyType', 'Fuel [note 2]', 'Keepership [note 3]', 'ONS Sort [note 6]', 
                                                    'ONS Code [note 6]', 'ONS Geography [note 6]'], var_name='Year', 
                                                    value_vars = ['2022 Q4', '2022 Q3', '2022 Q2', '2022 Q1', '2021 Q4', '2021 Q3', 
                                                                  '2021 Q2', '2021 Q1', '2020 Q4', '2020 Q3', '2020 Q2', '2020 Q1', 
                                                                  '2019 Q4', '2019 Q3', '2019 Q2', '2019 Q1', '2018 Q4', '2018 Q3', 
                                                                  '2018 Q2', '2018 Q1', '2017 Q4', '2017 Q3', '2017 Q2', '2017 Q1', 
                                                                  '2016 Q4', '2016 Q3', '2016 Q2', '2016 Q1', '2015 Q4', '2015 Q3', 
                                                                  '2015 Q2', '2015 Q1', '2014 Q4', '2014 Q3', '2014 Q2', '2014 Q1', 
                                                                  '2013 Q4', '2013 Q3', '2013 Q2', '2013 Q1', '2012 Q4', '2012 Q3', 
                                                                  '2012 Q2', '2012 Q1', '2011 Q4', '2011 Q3', '2011 Q2', '2011 Q1', 
                                                                  '2010 Q4', '2010 Q3', '2010 Q2', '2010 Q1', '2009 Q4'])
vehicle_fuel_df.head()

,Units,BodyType,Fuel [note 2],Keepership [note 3],ONS Sort [note 6],ONS Code [note 6],ONS Geography [note 6],Year,value
0,Thousands,Buses and coaches,Diesel,Company,1.0,K02000001,United Kingdom,2022 Q4,110.929
1,Thousands,Buses and coaches,Diesel,Company,2.0,K03000001,Great Britain,2022 Q4,106.768
2,Thousands,Buses and coaches,Diesel,Company,3.0,E92000001,England,2022 Q4,90.01
3,Thousands,Buses and coaches,Diesel,Company,4.0,E12000001,North East,2022 Q4,4.103
4,Thousands,Buses and coaches,Diesel,Company,5.0,E06000047,County Durham,2022 Q4,0.608


In [141]:
path = "data/veh0142.ods"

# load a sheet based on its index (1 based)
sheet_idx = 4
vehicle_electricity_df = read_ods(path, sheet_idx)

# make third row the header
vehicle_electricity_df.columns = vehicle_electricity_df.iloc[3]
# drop first 4 rows
vehicle_electricity_df = vehicle_electricity_df.iloc[4:]
vehicle_electricity_df.head()

# renumber index
vehicle_electricity_df = vehicle_electricity_df.reset_index(drop=True)
# melt the dataframe
vehicle_electricity_df = pd.melt(vehicle_electricity_df, id_vars=['Units', 'BodyType', 'Fuel', 'Keepership [note 3]', 
                                                                  'ONS Sort [note 6]', 'ONS Code [note 6]', 'ONS Geography [note 6]'], 
                                                                  var_name='Year', value_vars = ['2022 Q3', '2022 Q2', '2022 Q1', 
                                                                                                 '2021 Q4', '2021 Q3', '2021 Q2', 
                                                                                                 '2021 Q1', '2020 Q4', '2020 Q3', 
                                                                                                 '2020 Q2', '2020 Q1', '2019 Q4', 
                                                                                                 '2019 Q3', '2019 Q2', '2019 Q1', 
                                                                                                 '2018 Q4', '2018 Q3', '2018 Q2', 
                                                                                                 '2018 Q1', '2017 Q4', '2017 Q3', 
                                                                                                 '2017 Q2', '2017 Q1', '2016 Q4', 
                                                                                                 '2016 Q3', '2016 Q2', '2016 Q1', 
                                                                                                 '2015 Q4', '2015 Q3', '2015 Q2', 
                                                                                                 '2015 Q1', '2014 Q4', '2014 Q3', 
                                                                                                 '2014 Q2', '2014 Q1', '2013 Q4', 
                                                                                                 '2013 Q3', '2013 Q2', '2013 Q1', 
                                                                                                 '2012 Q4', '2012 Q3', '2012 Q2', 
                                                                                                 '2012 Q1', '2011 Q4', '2011 Q3', 
                                                                                                 '2011 Q2', '2011 Q1', '2010 Q4', 
                                                                                                 '2010 Q3', '2010 Q2', '2010 Q1', 
                                                                                                 '2009 Q4'])
vehicle_electricity_df.head()

,Units,BodyType,Fuel,Keepership [note 3],ONS Sort [note 6],ONS Code [note 6],ONS Geography [note 6],Year,value
0,Number,Buses and coaches,Battery electric,Company,1.0,K02000001,United Kingdom,2022 Q3,1808.0
1,Number,Buses and coaches,Battery electric,Company,2.0,K03000001,Great Britain,2022 Q3,1702.0
2,Number,Buses and coaches,Battery electric,Company,3.0,E92000001,England,2022 Q3,1343.0
3,Number,Buses and coaches,Battery electric,Company,4.0,E12000001,North East,2022 Q3,21.0
4,Number,Buses and coaches,Battery electric,Company,5.0,E06000047,County Durham,2022 Q3,[c]


In [29]:
# load a sheet with car models bought and quarter they were purchased
car_models_df = pd.read_csv('data/vehicle-licensing-statistics-2022-data-files/df_VEH0120_GB.csv')
car_models_df.head()

,BodyType,Make,GenModel,Model,Fuel,LicenceStatus,2022Q4,2022Q3,2022Q2,2022Q1,...,2003Q4,2002Q4,2001Q4,2000Q4,1999Q4,1998Q4,1997Q4,1996Q4,1995Q4,1994Q4
0,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR,Petrol,Licensed,17,17,18,18,...,0,0,0,0,0,0,0,0,0,0
1,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR,Petrol,SORN,3,2,2,2,...,0,0,0,0,0,0,0,0,0,0
2,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR AUTO,Petrol,Licensed,24,27,26,26,...,0,0,0,0,0,0,0,0,0,0
3,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR AUTO,Petrol,SORN,3,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,Cars,ABARTH,ABARTH SPIDER,124 SPIDER MULTIAIR,Petrol,Licensed,982,996,1000,986,...,0,0,0,0,0,0,0,0,0,0


In [30]:
# get data with fuel prices
fuel_prices_df = pd.read_excel('data/Weekly_Fuel_Prices_120623.xlsx', sheet_name='All years', skiprows=7)
fuel_prices_df.head()

,Date,ULSP: Pump price (p/litre),ULSP: Diff on previous WEEK (p/litre),ULSP: Diff on previous YEAR (p/litre),Duty rate ULSP (p/litre),VAT (% rate) ULSP,ULSD: Pump price (p/litre),ULSD: Diff on previous WEEK (p/litre),ULSD: Diff on previous YEAR (p/litre),Duty rate ULSD (p/litre),VAT (% rate) ULSD
0,2003-06-09,74.590284,NaN,NaN,45.82,17.5,76.773394,NaN,NaN,45.82,17.5
1,2003-06-16,74.469143,-0.121141,NaN,45.82,17.5,76.689054,-0.084340,NaN,45.82,17.5
2,2003-06-23,74.423566,0.000000,NaN,45.82,17.5,76.620546,-0.068508,NaN,45.82,17.5
3,2003-06-30,74.352421,-0.071145,NaN,45.82,17.5,76.505260,-0.115286,NaN,45.82,17.5
4,2003-07-07,74.283737,-0.068684,NaN,45.82,17.5,76.457376,0.000000,NaN,45.82,17.5
